In [52]:
import csv
import pandas as pd
import numpy as np
import sys

from fancyimpute import MICE, BiScaler, KNN, NuclearNormMinimization, SoftImpute

from sklearn.linear_model import RandomizedLasso

debug = False

def drop_low_var_quality_features(df, missing_values_threshold, std_threshold):
    """
    Params:
    missing_values_threshold: Maximum number of missing values for a column to be retained
    std_threshold: Minimum standard deviation for a column to be retained.
    """
    rows_count, cols_count = df.shape
    df.dropna(axis=1, thresh=rows_count-missing_values_threshold, inplace=True)
    df.drop(df.std()[df.std() < std_threshold].index.values, axis=1, inplace=True)

def is_valid_type(values):
    validity_list = []
    for val in values:
        isValid = True if type(val) == bool or np.isfinite(val) else False
        validity_list.append(isValid)
    return validity_list
    
def get_data_for_characteristic(X_train, Y_train, characteristic, get_only_complete_cases=False):
    y_char = Y_train[is_valid_type(Y_train[characteristic])]
    
    training_ids = y_char['challengeID'].tolist()
    X_char = X_train[X_train['challengeID'].isin(training_ids)]
    X_char = X_char.sort_values(by='challengeID')
    y_char = y_char.sort_values(by='challengeID')
    
    non_numeric_cols = X_char.select_dtypes(exclude=[np.number]).columns.values.tolist()
    X_char.drop(non_numeric_cols, axis=1, inplace=True)
    
    if get_only_complete_cases is True:
        X_char = X_char.dropna(axis=0, inplace=False)
        training_ids = X_char['challengeID'].tolist()
        y_char = y_char[y_char['challengeID'].isin(training_ids)]

    assert(y_char['challengeID'].tolist() == X_char['challengeID'].tolist())
    y_char.index = y_char['challengeID'] 
    X_char.index = X_char['challengeID']
    return X_char, y_char[characteristic]

In [53]:
def perform_imputation_for_characteristic(df, y_train_df, selection_threshold, characteristic, imputeKNN=False):
    # Get Xs and corresponding Ys for a characteristic. Only get the complete Xs (without any nan values)
    X_char_complete, y_char_complete = get_data_for_characteristic(df, y_train_df, characteristic, get_only_complete_cases=True)
    
    # Convert the complete cases DFs to Matrix
    X_char_c_mat, y_char_c_mat = X_char_complete.as_matrix(), y_char_complete.as_matrix()
    
    # Perform Randomized Lasso feature selection using the complete cases.
    lasso = RandomizedLasso(alpha='aic', random_state=39, n_resampling=500)
    lasso.fit(X_char_c_mat, y_char_c_mat)
    stability_scores = lasso.scores_
    support = np.where(stability_scores > selection_threshold)[0]
    
    print selection_threshold, '\t', str(len(support))
    if debug:
        print str(support)
    
    # Shrink the the X's dataframe to only required features
    XFFull = df.iloc[:,support]
    
    # Perform the imputation
    XFFull_mat = XFFull.as_matrix()
    try:
        if imputeKNN:
            X_full_imputed = KNN(k=3).complete(XFFull_mat)
        else:
            X_full_imputed = MICE().complete(XFFull_mat)
    except ValueError:
        X_full_imputed = XFFull_mat
        print "Input matrix is not missing any values."
    
    print "NaN's before imputation {}, after {}.".format(np.count_nonzero(np.isnan(XFFull_mat)), np.count_nonzero(np.isnan(X_full_imputed)))
    print "NaN was at {}".format(np.argwhere(np.isnan(XFFull_mat)))
    
    return pd.DataFrame(X_full_imputed, index=XFFull.index, columns=XFFull.columns)

In [3]:
df = pd.read_csv('background.csv', low_memory=False)
df.cf4fint = ((pd.to_datetime(df.cf4fint) - pd.to_datetime('1960-01-01')) / np.timedelta64(1, 'D')).astype(int)
df.index = df['challengeID']

if debug:
    # number of nan cols
    print sum(df.isnull().sum().tolist())
    # number of non-nan cols
    print df.count().sum()
    # Total elements
    print df.size
    # Shape
    print df.shape

drop_low_var_quality_features(df, 200, 0.2)

if debug:
    print sorted(df.isnull().sum().tolist(), reverse=True)[:10]
    print sorted(df.std())[:30]
    print df.shape

y_train_df = pd.read_csv("train.csv", low_memory=False)

In [59]:
imputed_pd = perform_imputation_for_characteristic(df, y_train_df, .00001, 'layoff')

1e-05 	73
[MICE] Completing matrix with shape (4242, 73)
Input matrix is not missing any values.
NaN's before imputation 0, after 0.
NaN was at []


In [26]:
y_train_df

,challengeID,gpa,grit,materialHardship,eviction,layoff,jobTraining
0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN
2,6,NaN,3.50,0.090909,False,False,False
3,7,2.50,3.25,0.000000,False,False,False
4,8,NaN,NaN,NaN,NaN,NaN,NaN
5,9,2.25,4.00,0.181818,False,False,False
6,10,3.25,3.25,0.090909,False,NaN,False
7,13,2.75,4.00,0.181818,False,False,True
8,14,3.25,2.75,0.272727,False,True,False
9,16,2.00,3.50,0.090909,False,False,True


In [55]:
imputed_pd.iloc[21, 6]

-6.1502726644532508